In [38]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [39]:
# 밴딧의 손잡이 목록을 작성한다.
# 현재 손잡이 4 (인덱스는 3)가 가장 자주 양의 보상을 제공하도록 설정되어 있다.
bandit_arms = [0.2, 0.0, -0.2, -2.0]
num_arms = len(bandit_arms)

In [40]:
def pullBandit(bandit):
    # 랜덤한 값을 구한다
    result = np.random.randn(1)
    
    if result > bandit:
        # 양의 보상을 반환한다
        return 1
    else:
        # 음의 보상을 반환한다
        return -1

In [41]:
# 간단한 신경망 구현
# 신경망은 각 밴딧 손잡이에 대한 일련의 값들로 구성
# 각 값은 해당 밴딧을 선택할 때 반환되는 보상의 추정값을 의미
# 정책 경사 방법을 이용해 선택된 액션에 대해 큰 보상을 받는 쪽으로 이동해나가며 에이전트를 업데이트
tf.reset_default_graph()

# 네트워크의 피드포워드 부분 구현
weights = tf.Variable(tf.ones([num_arms]))
# tf.ones([num_arms]) => 모든 요소가 1로 이루어진 shape가 4인 텐서 생성 [1, 1, 1, 1]
output = tf.nn.softmax(weights)
# weights의 값들의 비율을 총합이 1인 값으로 변환
# 각 Arm을 선택할 확률이 저장됨.
# 차후 학습을 통해 weights 값이 변경되고 그 결과가 tfModel이 된다.

In [42]:
# 테스트용 셀 - 학습에 영향을 미치지 않음

tRes = tf.Session().run(tf.ones([num_arms]))
print(tRes)
resOutput =tf.Session().run(tf.nn.softmax(tRes))
print(resOutput)

a = np.random.choice(resOutput, p=resOutput)
print("Selected Action: ")
print(a)
b = np.argmax(actions == a)
print(b)

[1. 1. 1. 1.]
[0.25 0.25 0.25 0.25]
Selected Action: 
0.25
0


In [43]:
# 테스트용 셀 - 학습에 영향을 미치지 않음

x = np.array([1, 3, 5, 4, 2, 6])
print(f'np.argmax(x == 5): {np.argmax(x == 5)}')
print(np.argmax(x))

np.argmax(x == 5): 2
5


In [44]:
# 학습 과정을 구현한다
# 보상과 선택된 액션을 네트워크에 피드해줌으로써 비용을 계산하고 비용을 이용해 네트워크를 업데이트한다
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(output, action_holder, [1])
# tf.slice는 다차원 배열 (또는 텐서)를 잘라내어 새로운 배열(텐서)를 생성하는 함수
# 관련 내용은 https://pythonkim.tistory.com/65 여기 참조
# 위 코드는 output에 저장된 텐서에서 action_holder에 저장된 특정 액션에 해당하는 값을 한개의 요소만 갖는 1차원 텐서로 뽑아내라는 뜻

loss = -1 * (tf.log(responsible_output) * reward_holder)
# 위 코드에서 reward_holder에는 responsible_output 값을 생성한 액션에 해당하는 보상값이 저장될 예정 (차후 돌아갈 sess 에서.)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

In [45]:
# 에이전트를 학습시킬 총 에피소드의 수를 설정한다
total_episodes = 1000
# 밴딧 손잡이에 대한 점수판을 0으로 설정
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

In [47]:
# 텐서플로 그래프를 론칭한다
with tf.Session() as sess:
    sess.run(init)
    
    i = 0
    while i < total_episodes:
        # 볼츠만 분포에 따라 액션 선택
        actions = sess.run(output)
        
        a = np.random.choice(actions, p=actions)
        # actions 배열 중에서 하나의 요소를 랜덤으로 고르는 함수, p 파라미터로 념겨준 겂은 각 요소를 선택하는 데 대한 확률 값
        
        action = np.argmax(actions == a)
        # np.argmax(actions)는 actions 배열 중에서 가장 큰 값의 위치(인덱스 값)를 반환한다.
        # np.argmax(actions == a) 는 actions 배열 중 a와 동일한 값의 위치를 반환 (요소 별 연산을 수행)
        
        # 위 로직을 수행하면, 
        # weight의 값을 총합 1인 값으로 정규화한 배열을 받아온다.
        # 그리고 그 값들을 확률로 반영해서 랜덤으로 하나의 값을 선택
        # 해당 값의 인덱스값 => Arm 번호를 받아온다.
        
        # 밴딧 손잡이 중 하나를 선택함으로써 보상을 받는다
        reward = pullBandit(bandit_arms[action])
        
        # 네트워크를 업데이트한다
        _, resp, ww = sess.run([update, responsible_output, weights], feed_dict={
            reward_holder:[reward], 
            action_holder: [action]
        })
        
        # 보상의 총계 업데이트
        total_reward[action] += reward
        if i % 50 == 0:
            print(f"Available Actions: {actions}")
            print(f"Selected Action: {a}")
            print(f"Updated Resp: {resp}")
            print(f"Updated weights: {ww}")
            print(f"\tRunning reward for the {num_arms} arms of the bandit: {total_reward}")
        i += 1

print("\nThe agent thinks arm")
        

Available Actions: [0.25 0.25 0.25 0.25]
Selected Action: 0.25
Updated Resp: [0.25]
Updated weights: [1.001 0.999 0.999 0.999]
	Running reward for the 4 arms of the bandit: [-19.  12.  70. 300.]
Available Actions: [0.24686424 0.24919233 0.24810286 0.25584057]
Selected Action: 0.24686424434185028
Updated Resp: [0.24686424]
Updated weights: [0.9862316  0.99624294 0.9919169  1.0234915 ]
	Running reward for the 4 arms of the bandit: [-27.  12.  69. 315.]
Available Actions: [0.24227208 0.24776036 0.2467943  0.2631732 ]
Selected Action: 0.24227207899093628
Updated Resp: [0.24227208]
Updated weights: [0.96736145 0.9902406  0.9862419  1.0512754 ]
	Running reward for the 4 arms of the bandit: [-30.  14.  71. 332.]
Available Actions: [0.23837377 0.24564317 0.24602653 0.2699565 ]
Selected Action: 0.24602653086185455
Updated Resp: [0.24602653]
Updated weights: [0.9506854  0.9812165  0.98353696 1.076016  ]
	Running reward for the 4 arms of the bandit: [-32.  15.  78. 348.]
Available Actions: [0.233